<a href="https://colab.research.google.com/github/Naufal-sul/CMS1/blob/main/CoLab_Torrent_Downloader_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install python3-libtorrent


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libtorrent-rasterbar2.0
The following NEW packages will be installed:
  libtorrent-rasterbar2.0 python3-libtorrent
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 2,098 kB of archives.
After this operation, 7,907 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtorrent-rasterbar2.0 amd64 2.0.5-5 [1,501 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-libtorrent amd64 2.0.5-5 [597 kB]
Fetched 2,098 kB in 0s (6,200 kB/s)
Selecting previously unselected package libtorrent-rasterbar2.0:amd64.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../libtorrent-rasterbar2.0_2.0.5-5_amd64.deb ...
Unpacking libtorrent-rasterbar2.0:amd64 (2.0.5-5) ...
Selecting previously unselected package python3-li

In [2]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.9 MB/s  0:00:00


In [8]:
import libtorrent as lt
import time
import re
import os
import pickle
from google.colab import drive, files
from tqdm import tqdm
import patoolib
from pathlib import Path

drive.mount('/content/drive')

ses = lt.session()
ses.listen_on(6881, 6891)

save_path = '/content/drive/MyDrive/Torrent Downloads/'

params = {
    'save_path': save_path,
    'storage_mode': lt.storage_mode_t.storage_mode_sparse,
}

magnet_link_pattern = r'^magnet:\?xt=urn:btih:[a-zA-Z0-9]*'

resume_data_file = os.path.join(save_path, 'resume_data.pickle')

if os.path.exists(resume_data_file):
    with open(resume_data_file, 'rb') as f:
        resume_data = pickle.load(f)
        for data in resume_data:
            ses.add_torrent(data)

while True:
    print("\nMenu:")
    print("1. Enter Magnet link: ")
    print("2. Upload Torrent file: ")
    print("3. File Options")
    print("4. Exit")

    choice = input("Select an option (1, 2, 3 or 4): ")

    if choice == '1':
        print()
        link = input("Enter the magnet link: ")
        if re.match(magnet_link_pattern, link):
            handle = lt.add_magnet_uri(ses, link, params)
        else:
            print("Invalid magnet link format. Please try again.")
            continue
    elif choice == '2':
        uploaded_files = files.upload()
        if len(uploaded_files) == 0:
            print("No files uploaded. Please try again.")
            continue

        torrent_file_path = list(uploaded_files.values())[0]

        try:
            handle = ses.add_torrent({'ti': lt.torrent_info(torrent_file_path), 'save_path': save_path})
        except Exception as e:
            print(f"Error: {e}")
            continue
    elif choice == '3':
      if 'handle' in locals():
          while True:
              print("\nFile Options:")
              print("1. Show File Details")
              print("2. Zip Manager")
              print("3. Exit to Main Menu")

              file_choice = input("Select an option (1, 2 or 3): ")

              if file_choice == '1':
                  for i in range(torrent_info.num_files()):
                      file_path = os.path.join(save_path, torrent_info.files().file_path(i))
                      if os.path.exists(file_path):
                          file_size = os.path.getsize(file_path)
                          file_size_mb = file_size / (1024 ** 2)
                          file_size_gb = file_size / (1024 ** 3)
                          file_size_str = f"{file_size_mb:.2f} MB / {file_size_gb:.2f} GB"
                          print(f"\nFile name: {torrent_info.files().file_path(i)}")
                          print(f"File size: {file_size_str}")
                          print(f"File link: {link}")
              elif file_choice == '2':
                  while True:
                      print("\nZip Manager:")
                      print("1. Extract ZIP archive")
                      print("2. Create ZIP archive")
                      print("3. Exit to File Options\n")

                      zip_choice = input("Select an option (1, 2 or 3): ")

                      if zip_choice == '1':
                          zip_file_path = input("\nEnter the path of the ZIP archive to extract: ")
                          extract_dir = input("Enter the directory where the archive should be extracted: ")
                          try:
                              patoolib.extract_archive(zip_file_path, outdir=extract_dir)
                              print(f"\nZIP archive extracted to {extract_dir}")
                          except Exception as e:
                              print(f"\nError: {e}")
                      elif zip_choice == '2':
                          # Get a list of all files and directories in the specified path
                          files_and_dirs = [f for f in os.listdir(save_path)]

                          # Display all files and directories in a numbered fashion
                          for i, f in enumerate(files_and_dirs):
                              print(f"{i+1}. {f}")

                          # Get user input for the indices of the files or directories to zip
                          files_to_zip_indices = input("\nEnter the numbers of the files or directories to add to the ZIP archive (separated by commas): ").split(',')
                          files_to_zip_indices = [int(i)-1 for i in files_to_zip_indices]

                          # Get the list of files or directories to zip
                          files_to_zip = [files_and_dirs[i] for i in files_to_zip_indices]

                          # Get user input for the name of the ZIP archive
                          zip_file_name = input("Enter the name of the ZIP archive to create: ")

                          # Create a list of file paths to add to the ZIP archive
                          file_paths = []
                          for file_or_dir in files_to_zip:
                              file_or_dir_path = os.path.join(save_path, file_or_dir)
                              if os.path.isfile(file_or_dir_path):
                                  file_paths.append(file_or_dir_path)
                              elif os.path.isdir(file_or_dir_path):
                                  for root, dirs, files in os.walk(file_or_dir_path):
                                      for file in files:
                                          file_path = os.path.join(root, file)
                                          file_paths.append(file_path)

                          # Create the ZIP archive
                          try:
                              patoolib.create_archive(zip_file_name, file_paths)
                              print(f"\nZIP archive created: {zip_file_name}")
                          except Exception as e:
                              print(f"\nError: {e}")
                      elif zip_choice == '3':
                          break
                      else:
                          print("Invalid choice. Please select a valid option.")
                          continue
              elif file_choice == '3':
                  break
              else:
                  print("Invalid choice. Please select a valid option.")
                  continue
      else:
          print("\nNo file has been downloaded yet.")
          continue
    elif choice == '4':
        print("\nExiting the program.")
        break
    else:
        print("Invalid choice. Please select a valid option.")
        continue

    if choice == '1' or choice == '2':
      print("\nDownloading Metadata...")
      while not handle.has_metadata():
          time.sleep(1)
      print("Got Metadata, Starting Torrent Download...")

      torrent_info = handle.get_torrent_info()

      for i in range(torrent_info.num_files()):
          handle.file_priority(i, 1)

      file_size = torrent_info.total_size()
      file_size_mb = file_size / (1024 ** 2)
      file_size_gb = file_size / (1024 ** 3)
      file_size_str = f"{file_size_mb:.2f} MB / {file_size_gb:.2f} GB"
      print(f"File size: {file_size_str}")

      progress_bar = tqdm(total=file_size, unit='B', unit_scale=True, leave=True)

      start_time = time.time()

      while handle.status().state != lt.torrent_status.seeding:
          s = handle.status()
          state_str = ['queued', 'checking', 'downloading metadata', 'downloading', 'finished', 'seeding', 'allocating', 'checking fastresume']
          progress_bar.set_description(state_str[min(s.state, len(state_str)-1)])
          progress_bar.update(s.total_done - progress_bar.n)
          time.sleep(1)

      progress_bar.close()

      end_time = time.time()
      total_time_seconds = end_time - start_time
      total_time_minutes = total_time_seconds / 60

      num_files = torrent_info.num_files()

      print(f"\nFile name: {torrent_info.name()}")
      print(f"File size: {file_size_str}")
      print(f"Total Time: {total_time_seconds:.2f} seconds / {total_time_minutes:.2f} minutes")
      print(f"Number of files: {num_files}")
      print(f"Saved location: {save_path}")

# save resume data
resume_data = []
for handle in ses.get_torrents():
    data = handle.save_resume_data()
    if data:
        resume_data.append({'resume_data': data, 'ti': handle.get_torrent_info(), 'save_path': save_path})
with open(resume_data_file, 'wb') as f:
    pickle.dump(resume_data, f)

Mounted at /content/drive

Menu:
1. Enter Magnet link: 
2. Upload Torrent file: 
3. File Options
4. Exit
Select an option (1, 2, 3 or 4): 2


Saving CF05F97D3A285A1422A8540053DE0FF77FE6D1B6.torrent to CF05F97D3A285A1422A8540053DE0FF77FE6D1B6.torrent
Error: 'utf-8' codec can't decode byte 0xdf in position 5460: invalid continuation byte

Menu:
1. Enter Magnet link: 
2. Upload Torrent file: 
3. File Options
4. Exit
Select an option (1, 2, 3 or 4): 1

Enter the magnet link: magnet:?xt=urn:btih:CF05F97D3A285A1422A8540053DE0FF77FE6D1B6&dn=The.Wonderfully.Weird.World.Of.Gumball.S01.REPACK.WEB.H264-RBB&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Fexodus.desync.com%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.cyberia.is%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce&tr=udp%3A%2F%2Fexplodie.org%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.birkenwald.de%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.moeking.me%3A6969%2Fannounce&tr=udp%3A%2F%2Fipv4.tracker.harry.lu%3A80%2Fannounce&tr=udp%3A%2F%2Fodd-hd.fr%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.jamest

downloading: 100%|█████████▉| 2.32G/2.32G [16:36<00:00, 2.33MB/s]



File name: The.Wonderfully.Weird.World.Of.Gumball.S01.REPACK.WEB.H264-RBB
File size: 2212.74 MB / 2.16 GB
Total Time: 996.55 seconds / 16.61 minutes
Number of files: 21
Saved location: /content/drive/MyDrive/Torrent Downloads/

Menu:
1. Enter Magnet link: 
2. Upload Torrent file: 
3. File Options
4. Exit
Select an option (1, 2, 3 or 4): 4

Exiting the program.


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip install patool